In [37]:
import json

import os
import yfinance as yf
import requests
from dotenv import load_dotenv
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.messages import ToolCall
from openai import OpenAI
from openai.lib.streaming import AssistantEventHandler
from openai.types.beta.threads.runs import RunStep, FunctionToolCall, ToolCallDelta
from openai.types.chat.chat_completion_message_tool_call import Function
from typing_extensions import override

client = OpenAI()

load_dotenv()


# openai assistant playground 에서는 실행 결과를 직접 붙여 넣기 해야 한다
def get_ticker(inputs):
    # return 'CRM'
    company_name = inputs['company_name']
    ddg = DuckDuckGoSearchAPIWrapper()
    return ddg.run(f'ticker symbol for {company_name}')


def get_income_statement(inputs):
    ticker = inputs['ticker']
    company = yf.Ticker(ticker)
    pd_frame = company.income_stmt
    json_obj = pd_frame.to_json(
        orient='records',  # Output format: list of records
        date_format='iso',  # Use ISO date format
        double_precision=2,  # Precision of floating-point numbers
        force_ascii=False,  # Preserve non-ASCII characters
        date_unit='ms'  # Timestamp unit in milliseconds
    )

    return json.dumps(json_obj)


def get_balance_sheet(inputs):
    ticker = inputs['ticker']
    company = yf.Ticker(ticker)
    pd_frame = company.balance_sheet
    json_obj = pd_frame.to_json(
        orient='records',  # Output format: list of records
        date_format='iso',  # Use ISO date format
        double_precision=2,  # Precision of floating-point numbers
        force_ascii=False,  # Preserve non-ASCII characters
        date_unit='ms'  # Timestamp unit in milliseconds
    )

    return json.dumps(json_obj)


def get_daily_stock_performance(inputs):
    ticker = inputs['ticker']
    company = yf.Ticker(ticker)
    pd_frame = company.history(period='3mo')
    json_obj = pd_frame.to_json(
        orient='records',  # Output format: list of records
        date_format='iso',  # Use ISO date format
        double_precision=2,  # Precision of floating-point numbers
        force_ascii=False,  # Preserve non-ASCII characters
        date_unit='ms'  # Timestamp unit in milliseconds
    )

    return json.dumps(json_obj)


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

# open ai assistant 생성
# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help them decide if they should buy the stock or not",
#     model="gpt-4-turbo",
#     tools=functions
# )

assistant_id = 'asst_xJDilHdQyVS7L9jn8xODLqlh'

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy"
        }
    ]
)

class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id  # Retrieve the run ID from the event data
            self.handle_requires_action(event.data, run_id)

    def handle_requires_action(self, data, run_id):
        
        tool_outputs = []
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            function_name = tool.function.name
            function_args = json.loads(tool.function.arguments)
            tool_call_result = globals()[function_name](function_args)
            tool_outputs.append({"tool_call_id": tool.id, "output": tool_call_result})
        
        self.submit_tool_outputs(tool_outputs, run_id)    
        
    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
        for text in stream.text_deltas:
          print(text, end="", flush=True)
        print()

with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant_id,
        event_handler=EventHandler(),
) as stream:
    stream.until_done()                   
                    

### Analysis of Salesforce (CRM) Stock

#### Financial Overview
- **Revenue**: Salesforce has shown an increase in revenue year by year, indicating expanding business operations and market penetration.
- **Net Income**: The company faced a jarring net loss in the latest reported fiscal year but had posted profits in earlier years. This could raise concerns but is not uncommon in dynamic industries driven by heavy investments or acquisitions.
- **Assets vs. Liabilities**: Salesforce maintains a relatively healthy balance sheet with substantial assets to cover its liabilities, evidenced by its most recent financial data.

#### Recent Stock Performance
- Over the last 100 days, Salesforce's stock price has fluctuated significantly with highs and lows driven by various market factors and possibly internal company developments.

#### Short-term Trading View
- Stock performance over the past three months (100 days) has been volatile, indicating significant risks if you're considering a short